In [ ]:
from src import src,misc
import numpy as np
import matplotlib.pyplot as plt
pltr = misc.Plotter()

Load the observation matrix

In [ ]:
A = np.load('models/simulated/A15.npy') # Observation model matrix for 15 frequencies (cross array)
# Observation model matrix is of shape (k,t,r,x,y,z) 
# where:
# k = number of frequncy steps (i.e.: 15 for the saved matrix), 
# t = number of transmitter antennas (i.e.: 12 for the saved matrix), 
# r = number of receiver antennas (i.e.: 13 for the saved matrix),
# x,y,z = number of voxels at x,y,z dimensions (i.e.: 25,25,49)
print(A.shape)

Load the test data

In [ ]:
x=np.load('data/x_test.npy') # Complex-valued (random phase)

Simulate Measurements

In [ ]:
Ax = np.einsum('ktrxyz,Nxyz->Nktr',A,x) # Ax

np.random.seed(0)
y = misc.addNoise(signal=Ax,SNRdB=30) # y = Ax + n

Validate SNR

In [ ]:
print(np.mean([misc.computeSNRdB(signal=Ax[i,...],noise=y[i,...]-Ax[i,...]) for i in range(y.shape[0])]))

Load the reconstructor

In [ ]:
DNN=src.get_UNet3D()
DNN.load_weights('models/simulated/Deep2S/Unet3D/model_Nf15_SNR30.h5')
reconstructor = src.Deep2S(DNN=DNN,A=A)

Predict

In [ ]:
estimates = reconstructor.predict(y)

Compute Performance

In [ ]:
psnrs = misc.computePSNR(estimates,np.abs(x))
ssims = misc.computeSSIM(estimates.astype(float),np.abs(x).astype(float))

print('Average PSNR : {:.1f} dB, PSNR on the Sample Image: {:.2f} dB'.format(np.mean(psnrs),psnrs[4]))
print('Average SSIM : {:.2f}, SSIM on the Sample Image: {:.2f}'.format(np.mean(ssims),ssims[4]))

Plot the datacubes in 3D

In [ ]:
pltr.sceneshow(x=estimates[4,...],perspective='simulated').show()

Plot the datacubes in 2D

In [ ]:
plt.figure(figsize=(5,10))
pltr.maxProjection(x=estimates[4,...])